<a id='top'></a>
## Can Amazon Reviews Help Businesses Learn their Best Customers? 

#### Contributors: Muoyo Okome, Jill Carrie



[Back to Top ↑](#top)

<a id='toc'></a>
### Table of Contents
1. [Problem Statement](#problemstatement)
2. [Data Sources](#datasources)
3. [Data Cleaning](#datacleaning)
4. [Classification Analysis](#classification)  
    - [Logistic Regression](#logisticregression)
    - [K Nearest Neighbors](#kneighbors)
    - [Decision Trees](#decisiontrees)
    - [Bagged Trees](#baggedtrees)
    - [Random Forest](#randomforest)
    - [Adaboost](#adaboost)
    - [Gradient Boost](#gradientboost)
5. [Findings](#findings)
6. [Vizualizations](#findings)
7. [Next Steps](#nextsteps)

In [27]:
# Import necessary libraries
import warnings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
warnings.filterwarnings('ignore')
%matplotlib inline

<a id='problemstatement'></a>
### Problem Statement

**Can Amazon Reviews Help Businesses Learn their Best Customers?**

The key question we look to answer is whether knowing the text of an Amazon Customer Review can help us predict the star rating of that review.

Eventually our goal is to provide these insights to businesses as a service to help them identify customers most likely to love or hate their products based on the things they are saying.

[Back to Top ↑](#top)

<a id='datasources'></a>
### Data Sources

#### **[AWS Open Data: Amazon Customer Reviews Dataset](https://registry.opendata.aws/amazon:reviews/)**

- Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazon’s iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. Over 130+ million customer reviews are available to researchers as part of this dataset.

- A collection of reviews written in the Amazon.com marketplace and associated metadata from 1995 until 2015. This is intended to facilitate study into the properties (and the evolution) of customer reviews potentially including how people evaluate and express their experiences with respect to products at scale. (130M+ customer reviews) 

<br> 

[Back to Top ↑](#top)

In [21]:
# Import python files we've created to help
%run ../python_files/nlp
%run ../python_files/data_cleaning
%run ../python_files/classification
%run ../python_files/visualizations

In [5]:
# Read in original data
df = pd.read_csv('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz', sep='\t', error_bad_lines=False, warn_bad_lines=False)

<a id='datacleaning'></a>
### Data Cleaning

In preparation for our analysis, in addition to dealing with missing values and limiting our scope to the columns of interest, we also used [VADER Sentiment Analysis](https://github.com/cjhutto/vaderSentiment) (Valence Aware Dictionary and sEntiment Reasoner) to give us positive, negative, neutral, and compound sentiment scores for each review in our dataset. Finally, we leveraged SMOTE to deal with the issue of unbalanced classes in our data.

To make the project more modular and easier to follow & build upon, we created separate .py files to handle the heavy lifting for tasks such as data extraction & cleaning, visualizations, and natural language processing.

We chose to focus on the Watches category (~960,000 customer reviews) for this analysis, though with sufficient time, the process can be generalized to the entire dataset.

In [4]:
# Clean data & load into final dataframe

In [17]:
df = clean_data(df, 25000)
df = append_sentiment_scores(df)



[Back to Top ↑](#top)

<a id='classification'></a>
### Classification Analysis

With the help of the scikit-learn package, our next step was to run a series of classification models, each time with the goal of using insight gained from the review text, to predict the number of stars in the review's star rating.

- **Dependent variable:** **star_rating:** The 1-5 star rating of the review. <p>

- **Independendent variables:**
    - **helpful_votes:** Number of helpful votes the review received.
    - **total_votes:** Total number of votes the review received.
    - **neg:** Review's 'negative' rating via VADER Sentiment Analysis.
    - **neu:** Review's 'neutral' rating via VADER Sentiment Analysis.
    - **pos:** Review's 'positive' rating via VADER Sentiment Analysis.
    - **compound:** Review's 'compound' rating via VADER Sentiment Analysis.

To guard against data leakage, we split our dataset into training data and test data.

In [22]:
X_train, X_test, y_train, y_test = get_train_test_split(df, test_size=.25)



[Back to Top ↑](#top)

<a id='logisticregression'></a>
### Logistic Regression

In [32]:
clf_lr = LogisticRegression(fit_intercept=True, C=1e12, solver='liblinear', penalty='l2')
clf_lr.fit(X_train, y_train)

y_hat_lr_train = clf_lr.predict(X_train)

print(confusion_matrix(y_train, y_hat_lr_train), '\n\n')
print(classification_report(y_train, y_hat_lr_train))

[[6599 2220  811 1250  226]
 [4604 2988 1291 1919  304]
 [2710 2017 1417 3299 1663]
 [ 583 1008  767 4431 4317]
 [ 273  737  309 2979 6808]] 


              precision    recall  f1-score   support

           1       0.45      0.59      0.51     11106
           2       0.33      0.27      0.30     11106
           3       0.31      0.13      0.18     11106
           4       0.32      0.40      0.35     11106
           5       0.51      0.61      0.56     11106

    accuracy                           0.40     55530
   macro avg       0.38      0.40      0.38     55530
weighted avg       0.38      0.40      0.38     55530



In [41]:
y_hat_lr_test = clf_lr.predict(X_test)

print(confusion_matrix(y_test, y_hat_lr_test), '\n\n')
print(classification_report(y_test, y_hat_lr_test))

[[ 342  118   37   57   14]
 [ 157   76   33   65   23]
 [ 100   99   48  177   84]
 [  68  112   80  452  447]
 [  79  250  104  917 2311]] 


              precision    recall  f1-score   support

           1       0.46      0.60      0.52       568
           2       0.12      0.21      0.15       354
           3       0.16      0.09      0.12       508
           4       0.27      0.39      0.32      1159
           5       0.80      0.63      0.71      3661

    accuracy                           0.52      6250
   macro avg       0.36      0.39      0.36      6250
weighted avg       0.58      0.52      0.54      6250





[Back to Top ↑](#top)

<a id='kneighbors'></a>
### K Nearest Neighbors

In [33]:
clf_knn = KNeighborsClassifier()
clf_knn.fit(X_train, y_train)

y_hat_knn_train = clf_knn.predict(X_train)

print(confusion_matrix(y_train, y_hat_knn_train), '\n\n')
print(classification_report(y_train, y_hat_knn_train))

[[9376 1119  361  180   70]
 [ 872 9578  360  215   81]
 [ 638  940 8810  417  301]
 [ 420  718  726 8269  973]
 [ 505  940 1122 2011 6528]] 


              precision    recall  f1-score   support

           1       0.79      0.84      0.82     11106
           2       0.72      0.86      0.79     11106
           3       0.77      0.79      0.78     11106
           4       0.75      0.74      0.75     11106
           5       0.82      0.59      0.69     11106

    accuracy                           0.77     55530
   macro avg       0.77      0.77      0.76     55530
weighted avg       0.77      0.77      0.76     55530



In [42]:
y_hat_knn_test = clf_knn.predict(X_test)

print(confusion_matrix(y_test, y_hat_knn_test), '\n\n')
print(classification_report(y_test, y_hat_knn_test))

[[ 234  161  109   42   22]
 [ 105   93   93   34   29]
 [  92  117  125  109   65]
 [ 118  132  212  319  378]
 [ 169  346  428  886 1832]] 


              precision    recall  f1-score   support

           1       0.33      0.41      0.36       568
           2       0.11      0.26      0.15       354
           3       0.13      0.25      0.17       508
           4       0.23      0.28      0.25      1159
           5       0.79      0.50      0.61      3661

    accuracy                           0.42      6250
   macro avg       0.32      0.34      0.31      6250
weighted avg       0.55      0.42      0.46      6250





[Back to Top ↑](#top)

<a id='#decisiontrees'></a>
### Decision Trees

In [43]:
clf_dt = DecisionTreeClassifier(criterion='entropy')
clf_dt.fit(X_train, y_train)

y_hat_dt_train = clf_dt.predict(X_train)

print(confusion_matrix(y_train, y_hat_dt_train), '\n\n')
print(classification_report(y_train, y_hat_dt_train))

[[10390   690    21     1     4]
 [  113 10978     5     2     8]
 [   45   467 10535    12    47]
 [   23   297    33 10501   252]
 [   38   410    44   155 10459]] 


              precision    recall  f1-score   support

           1       0.98      0.94      0.96     11106
           2       0.85      0.99      0.92     11106
           3       0.99      0.95      0.97     11106
           4       0.98      0.95      0.96     11106
           5       0.97      0.94      0.96     11106

    accuracy                           0.95     55530
   macro avg       0.96      0.95      0.95     55530
weighted avg       0.96      0.95      0.95     55530



In [44]:
y_hat_dt_test = clf_dt.predict(X_test)

print(confusion_matrix(y_test, y_hat_dt_test), '\n\n')
print(classification_report(y_test, y_hat_dt_test))

[[ 208  147  103   56   54]
 [  97   85   77   45   50]
 [  71  104   97  117  119]
 [  77  121  167  289  505]
 [ 147  293  329  780 2112]] 


              precision    recall  f1-score   support

           1       0.35      0.37      0.36       568
           2       0.11      0.24      0.15       354
           3       0.13      0.19      0.15       508
           4       0.22      0.25      0.24      1159
           5       0.74      0.58      0.65      3661

    accuracy                           0.45      6250
   macro avg       0.31      0.32      0.31      6250
weighted avg       0.53      0.45      0.48      6250





[Back to Top ↑](#top)

<a id='#baggedtrees'></a>
### Bagged Trees

In [45]:
clf_bagged = BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_leaf_nodes=100), 
                                 n_estimators=20)
clf_bagged.fit(X_train, y_train)

y_hat_bagged_train = clf_bagged.predict(X_train)

print(confusion_matrix(y_train, y_hat_bagged_train), '\n\n')
print(classification_report(y_train, y_hat_bagged_train))

[[6232 2637 1573  544  120]
 [2329 5674 1991  842  270]
 [1619 2323 4165 1624 1375]
 [ 350 1268 1842 3620 4026]
 [ 222  801  945 2079 7059]] 


              precision    recall  f1-score   support

           1       0.58      0.56      0.57     11106
           2       0.45      0.51      0.48     11106
           3       0.40      0.38      0.39     11106
           4       0.42      0.33      0.37     11106
           5       0.55      0.64      0.59     11106

    accuracy                           0.48     55530
   macro avg       0.48      0.48      0.48     55530
weighted avg       0.48      0.48      0.48     55530



In [46]:
y_hat_bagged_test = clf_bagged.predict(X_test)

print(confusion_matrix(y_test, y_hat_bagged_test), '\n\n')
print(classification_report(y_test, y_hat_bagged_test))

[[ 266  183   72   29   18]
 [ 115  109   66   36   28]
 [  77  121  114  107   89]
 [  46  139  194  288  492]
 [  61  292  312  625 2371]] 


              precision    recall  f1-score   support

           1       0.47      0.47      0.47       568
           2       0.13      0.31      0.18       354
           3       0.15      0.22      0.18       508
           4       0.27      0.25      0.26      1159
           5       0.79      0.65      0.71      3661

    accuracy                           0.50      6250
   macro avg       0.36      0.38      0.36      6250
weighted avg       0.57      0.50      0.53      6250





[Back to Top ↑](#top)

<a id='randomforest'></a>
### Random Forest

In [47]:
clf_forest = RandomForestClassifier(n_estimators=100, max_depth = 10)
clf_forest.fit(X_train, y_train)

y_hat_forest_train = clf_forest.predict(X_train)

print(confusion_matrix(y_train, y_hat_forest_train), '\n\n')
print(classification_report(y_train, y_hat_forest_train))

[[7266 2217 1024  512   87]
 [1769 7471 1015  656  195]
 [1347 1904 5517 1203 1135]
 [ 381 1246 1179 5298 3002]
 [ 268  835  815 1905 7283]] 


              precision    recall  f1-score   support

           1       0.66      0.65      0.66     11106
           2       0.55      0.67      0.60     11106
           3       0.58      0.50      0.53     11106
           4       0.55      0.48      0.51     11106
           5       0.62      0.66      0.64     11106

    accuracy                           0.59     55530
   macro avg       0.59      0.59      0.59     55530
weighted avg       0.59      0.59      0.59     55530



In [49]:
y_hat_forest_test = clf_forest.predict(X_test)

print(confusion_matrix(y_test, y_hat_forest_test), '\n\n')
print(classification_report(y_test, y_hat_forest_test))

[[ 278  177   66   33   14]
 [ 129  106   55   37   27]
 [  79  125   98  120   86]
 [  68  135  175  320  461]
 [  93  285  271  708 2304]] 


              precision    recall  f1-score   support

           1       0.43      0.49      0.46       568
           2       0.13      0.30      0.18       354
           3       0.15      0.19      0.17       508
           4       0.26      0.28      0.27      1159
           5       0.80      0.63      0.70      3661

    accuracy                           0.50      6250
   macro avg       0.35      0.38      0.36      6250
weighted avg       0.57      0.50      0.53      6250





[Back to Top ↑](#top)

<a id='adaboost'></a>
### Adaboost

In [38]:
clf_ab = AdaBoostClassifier()
clf_ab.fit(X_train, y_train)

y_hat_ab_train = clf_ab.predict(X_train)

print(confusion_matrix(y_train, y_hat_ab_train), '\n\n')
print(classification_report(y_train, y_hat_ab_train))

[[5441 3745 1169  624  127]
 [3293 4696 1621 1291  205]
 [1783 3118 2300 2418 1487]
 [ 415 1618 1364 3398 4311]
 [ 228  898  633 2398 6949]] 


              precision    recall  f1-score   support

           1       0.49      0.49      0.49     11106
           2       0.33      0.42      0.37     11106
           3       0.32      0.21      0.25     11106
           4       0.34      0.31      0.32     11106
           5       0.53      0.63      0.57     11106

    accuracy                           0.41     55530
   macro avg       0.40      0.41      0.40     55530
weighted avg       0.40      0.41      0.40     55530



In [50]:
y_hat_ab_test = clf_ab.predict(X_test)

print(confusion_matrix(y_test, y_hat_ab_test), '\n\n')
print(classification_report(y_test, y_hat_ab_test))

[[ 265  199   50   34   20]
 [ 111  135   45   35   28]
 [  72  143   71  126   96]
 [  55  159  132  341  472]
 [  87  275  211  708 2380]] 


              precision    recall  f1-score   support

           1       0.45      0.47      0.46       568
           2       0.15      0.38      0.21       354
           3       0.14      0.14      0.14       508
           4       0.27      0.29      0.28      1159
           5       0.79      0.65      0.72      3661

    accuracy                           0.51      6250
   macro avg       0.36      0.39      0.36      6250
weighted avg       0.58      0.51      0.54      6250





[Back to Top ↑](#top)

<a id='gradientboost'></a>
### Gradient Boost

In [51]:
clf_gb = GradientBoostingClassifier()
clf_gb.fit(X_train, y_train)

y_hat_gb_train = clf_gb.predict(X_train)

print(confusion_matrix(y_train, y_hat_gb_train), '\n\n')
print(classification_report(y_train, y_hat_gb_train))

[[5992 3111 1361  523  119]
 [2372 5867 1620 1042  205]
 [1640 2748 3556 1885 1277]
 [ 344 1518 1662 4080 3502]
 [ 222  871  854 2253 6906]] 


              precision    recall  f1-score   support

           1       0.57      0.54      0.55     11106
           2       0.42      0.53      0.47     11106
           3       0.39      0.32      0.35     11106
           4       0.42      0.37      0.39     11106
           5       0.58      0.62      0.60     11106

    accuracy                           0.48     55530
   macro avg       0.47      0.48      0.47     55530
weighted avg       0.47      0.48      0.47     55530



In [52]:
y_hat_gb_test = clf_gb.predict(X_test)

print(confusion_matrix(y_test, y_hat_gb_test), '\n\n')
print(classification_report(y_test, y_hat_gb_test))

[[ 274  190   56   27   21]
 [ 116  119   62   29   28]
 [  73  142   95  110   88]
 [  42  168  177  309  463]
 [  67  298  260  757 2279]] 


              precision    recall  f1-score   support

           1       0.48      0.48      0.48       568
           2       0.13      0.34      0.19       354
           3       0.15      0.19      0.16       508
           4       0.25      0.27      0.26      1159
           5       0.79      0.62      0.70      3661

    accuracy                           0.49      6250
   macro avg       0.36      0.38      0.36      6250
weighted avg       0.57      0.49      0.52      6250





[Back to Top ↑](#top)

### Ordinary Least Squares via statsmodels

With our preprocessing complete (we conduct it within the **get_train_test_split()** function, as seen above), we are ready to run our series of regression. 

First, we run an OLS regression using statsmodels and learn that our model explains 26-30% of the variance of our independent variable **trip_total**, depending on the specific set of data we are working with.

Looking at the p-values for the coefficients, we can see that most of the variables are significant. 

However, our model does suffer from issues with the Jarque-Bera test, as well as the F-statistic, indicating that errors are not likely to be normally distributed, and the homoscedasticity assumption is likely to be violated. We need to focus on addressing these issues in our future analysis, perhaps using log transformations as a starting point.

[Back to Top ↑](#top)

<a id='findings'></a>
### Findings

1. **Drivers can earn the most at 5 am.** Peak fares occur between 4-6am on a daily basis. Across all the 7 days of the week, the peak earning time for drivers is always between 4-6 am. Factors that can help explain this trend is the fact that to earn more drivers are looking at driving long distances in less amount of time, early mornings provide an opportunity for long distances mostly to airports and relatively less traffic hence distance can be covered in shorter periods of time.<p>

2. **Airport pickups lead to higher earnings.** The 2 airports in Chicago are by far the highest earning pickup community areas, Neighborhoods that are far from the airport and with limited public transport lead the rest of the other community areas.<p>

3. **Day of week alone does not significantly affect expected fare.** Average fares tend to fall in the same range when viewed at the daily level. We need to combine with other predictors to get a better sense of the true relationships.<p>

4. **Strong relationship between temperature and trip fare. Of the independent variables used, Temperature has the strongest correlation with Fares. The colder it gets the more average fares increase**<p>



[Back to Top ↑](#top)

<a id='visualizations'></a>
### Visualizations

Let's further illustrate our findings via the illustrations below.

### Peak fares occur between 4am - 6am

### Airport pickups yield the highest fares



### Day of week alone does not significantly affect expected fare


[Back to Top ↑](#top)

<a id='nextsteps'></a>
### Next Steps

1. **Further refine the model to provide recommendations of ideal pickup location given time & weather.** To be able to provide drivers realtime predictions of where and when to be in order to earn the most, using their location, weather conditions, time and day of the week.<p>

2. **Finding additional and more correct independent variables to increase models predictive power** that may include events and activities going on in Chicago, emergencies, data on airport arrivals and delays, etc.<p>

3. **Extend our model to help different groups such as riders & competing rideshare companies.** Can we design models that can help riders to determine when a fee will drop and by how much to help them make better decisions of when to take a ride that will save them the most money?<p>



[Back to Top ↑](#top)